In [1]:
#-*- coding:utf-8
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Convolution1D, Conv2D, MaxPooling2D, MaxPooling1D, LSTM, Embedding
from keras.optimizers import SGD
import keras.backend as K
import random
import re
import tensorflow as tf
from keras.layers import *
from keras.models import *
from keras.utils import np_utils
from keras.regularizers import l2
from sklearn.model_selection import train_test_split

config_proto = tf.ConfigProto(log_device_placement=0,allow_soft_placement=0)
config_proto.gpu_options.allow_growth = True

Using TensorFlow backend.


In [2]:
X_train = np.load('../ECGdata/data1.npy')
temp = np.load('../ECGdata/data2.npy')
X_train = np.vstack((X_train,temp))
temp = np.load('../ECGdata/data3.npy')
X_train = np.vstack((X_train,temp))
Y_train = np.load('../ECGdata/label.npy')
print X_train.shape
print Y_train.shape

(6400, 216000)
(6400, 9)


In [3]:
x_train, x_valid, y_train, y_valid = train_test_split(X_train[:,-18000:], Y_train, test_size=0.2, random_state=2018)
print x_train.shape

(5120, 18000)


In [4]:
class Attention_layer(Layer):
    """
        Attention operation, with a context/query vector, for temporal data.
        Supports Masking.
        Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
        "Hierarchical Attention Networks for Document Classification"
        by using a context vector to assist the attention
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(AttentionWithContext())
        """
 
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
 
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(Attention_layer, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
 
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
 
        super(Attention_layer, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None
 
    def call(self, x, mask=None):
        uit = K.dot(x, self.W)
 
        if self.bias:
            uit += self.b
 
        uit = K.tanh(uit)
 
        a = K.exp(uit)
 
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
 
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        print a
        # a = K.expand_dims(a)
        print x
        weighted_input = x * a
        print weighted_input
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return (-1, input_shape[-1])

In [20]:
def conv_block(input, nb_filter, dropout_rate=None, weight_decay=1E-4):
    x = Activation('relu')(input)
    x = Convolution1D(nb_filter, (9), kernel_initializer="he_uniform", padding="same", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(x)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)
    return x

def dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1E-4):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    feature_list = [x]

    for i in range(nb_layers):
        x = conv_block(x, growth_rate, dropout_rate, weight_decay)
        feature_list.append(x)
        x = Concatenate(axis=concat_axis)(feature_list)
        nb_filter += growth_rate

    return x, nb_filter

def transition_block(input, nb_filter, dropout_rate=None, weight_decay=1E-4):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    x = Convolution1D(nb_filter, (1), kernel_initializer="he_uniform", padding="same", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(input)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)
    x = AveragePooling1D((2), strides=(1))(x)

    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay),
                           beta_regularizer=l2(weight_decay))(x)

    return x

def createDenseNet(nb_classes, img_dim, depth=40, nb_dense_block=3, growth_rate=12, nb_filter=16, dropout_rate=None,
                     weight_decay=1E-4, verbose=True):

    model_input = Input(shape=img_dim)

    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    assert (depth - 4) % 3 == 0, "Depth must be 3 N + 4"

    # layers in each dense block
    nb_layers = int((depth - 4) / 3)

    # Initial convolution
    x = Convolution1D(nb_filter, (9), kernel_initializer="he_uniform", padding="same", name="initial_conv2D", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(model_input)

    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay),
                            beta_regularizer=l2(weight_decay))(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate,
                                   weight_decay=weight_decay)
        # add transition_block
        x = transition_block(x, nb_filter, dropout_rate=dropout_rate, weight_decay=weight_decay)

    # The last dense_block does not have a transition_block
    x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate,
                               weight_decay=weight_decay)

    x = Activation('relu')(x)
#     print x.shape
#     x = AveragePooling2D((1, 25), strides=(1, 25))(x)
    x = GlobalAveragePooling1D()(x)
#     print x.shape
#     x = Attention_layer()(x)
    x = Dense(nb_classes, activation='softmax', kernel_regularizer=l2(weight_decay), bias_regularizer=l2(weight_decay))(x)

    densenet = Model(inputs=model_input, outputs=x)

    if verbose: 
        print("DenseNet-%d-%d created." % (depth, growth_rate))

    return densenet



#define DenseNet parms
ROWS = 12
COLS = 18000
CHANNELS = 1
nb_classes = 9
batch_size = 32
nb_epoch = 40
img_dim = (ROWS,COLS)
densenet_depth = 40
densenet_growth_rate = 12

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim,depth=densenet_depth,
                  growth_rate = densenet_growth_rate)

sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')
checkpoint = keras.callbacks.ModelCheckpoint('DenseNet.h5', monitor='val_acc', verbose=1, save_best_only=True)

xtrain = x_train.reshape(-1,12,18000)
xvalid = x_valid.reshape(-1,12,18000)
model.fit(xtrain, y_train, batch_size=32, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [checkpoint])

DenseNet-40-12 created.
Train on 5120 samples, validate on 1280 samples
Epoch 1/50
5120/5120 [==============================] - 24s 5ms/step - loss: 2.3376 - acc: 0.2092 - val_loss: 2.2991 - val_acc: 0.2461

Epoch 00001: val_acc improved from -inf to 0.24609, saving model to DenseNet.h5
Epoch 2/50
5120/5120 [==============================] - 17s 3ms/step - loss: 2.0533 - acc: 0.3314 - val_loss: 2.3413 - val_acc: 0.2180

Epoch 00002: val_acc did not improve from 0.24609
Epoch 3/50
5120/5120 [==============================] - 17s 3ms/step - loss: 1.8315 - acc: 0.4449 - val_loss: 2.4363 - val_acc: 0.2117

Epoch 00003: val_acc did not improve from 0.24609
Epoch 4/50
5120/5120 [==============================] - 17s 3ms/step - loss: 1.5759 - acc: 0.5613 - val_loss: 2.6736 - val_acc: 0.1648

Epoch 00004: val_acc did not improve from 0.24609
Epoch 5/50
5120/5120 [==============================] - 17s 3ms/step - loss: 1.3137 - acc: 0.6566 - val_loss: 2.9113 - val_acc: 0.2008

Epoch 00005: val_a

In [8]:
class F1(keras.layers.Layer):
    def __init__(self, classes, name='F1', **kwargs):        
        """
        classes: the output_dims
        """
        super(F1, self).__init__(name=name, **kwargs)
        self.classes = classes
        self.stateful = True
        self.zeros = np.zeros((classes, classes), dtype=np.float32)
        #something like confusion matrix
        self.confusion = K.variable(value=self.zeros, dtype='float32')
    pass

    def reset_states(self):
        """
        when the epoch ends, clear the confusion matrix
        """
        K.set_value(self.confusion, self.zeros)
        
    def __call__(self, y_true, y_pred):
        """
        when run the instant,call this function to calculate f1_score
        """
        true = K.one_hot(K.cast(K.max(y_true, axis=-1), dtype=np.uint8), self.classes)
        pred = K.one_hot(K.argmax(y_pred, axis=-1), self.classes)
        #calculate true.T * pred
        batch = K.dot(K.transpose(true), pred)
        self.add_update(K.update_add(self.confusion, batch), inputs=[y_true, y_pred])
        cur = self.confusion + batch
        #sum alongside column
        d1 = K.sum(cur, axis=0)
        #sum alongside row
        d2 = K.sum(cur, axis=1)
        # return the tensor's diagonal(对角线)
        diag = Lambda(tf.diag_part)(cur)
        return 2. * K.mean(diag / (d1 + d2 + 0.00001))
    
class AccF1MetricCallback(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.train_acc = []
        self.val_acc = []
        self.f1 = []

    def on_epoch_end(self, epoch, logs={}):
        self.train_acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        self.f1.append(logs.get('val_F1'))
#         val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
#         val_pred = np.zeros((val_predict.shape[0],1))
#         val_target = self.validation_data[1]
#         for i in range(val_predict.shape[0]):
#             val_pred[i,:] = np.argmax(val_predict[i,:])
#         _val_f1 = f1_score(val_target, val_predict)
#         self.f1.append(_val_f1)
#         f1_score = F1(CLASSES)
#         self.f1.append(f1_score(self.validation_data[1], self.model.predict(self.validation_data[0])))

    def plot_metric(self, mode):
        iterations = range(len(self.train_acc))
        plt.figure(figsize=(8,4))
        #acc
        plt.plot(iterations, self.train_acc, 'r', label='train_acc')
        #val_acc
        plt.plot(iterations, self.val_acc, 'g', label='val_acc')
        #f1
        plt.plot(iterations, self.f1, 'b', label='f1_score')
        plt.grid(True)
        ax = plt.axes()
        ax.set_xlabel('Epochs')
        ax.set_ylabel('Acc-F1')
        plt.legend(loc='lower right')
        plt.ylim((0.5,1))
        plt.show()
        return self.train_acc, self.val_acc, self.f1

def conv_block(input, nb_filter, dropout_rate=None, weight_decay=1E-4):
    x = Activation('relu')(input)
    x = Convolution2D(nb_filter, (1, 11), kernel_initializer="he_uniform", padding="same", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(x)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)
    return x

def dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1E-4):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    feature_list = [x]

    for i in range(nb_layers):
        x = conv_block(x, growth_rate, dropout_rate, weight_decay)
        feature_list.append(x)
        x = Concatenate(axis=concat_axis)(feature_list)
        nb_filter += growth_rate

    return x, nb_filter

def transition_block(input, nb_filter, dropout_rate=None, weight_decay=1E-4):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    x = Convolution2D(nb_filter, (1, 1), kernel_initializer="he_uniform", padding="same", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(input)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)
    x = AveragePooling2D((1, 2), strides=(1, 2))(x)

    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay),
                           beta_regularizer=l2(weight_decay))(x)

    return x

def createDenseNet(nb_classes, img_dim, depth=40, nb_dense_block=3, growth_rate=12, nb_filter=16, dropout_rate=None,
                     weight_decay=1E-4, verbose=True):

    model_input = Input(shape=img_dim)

    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    assert (depth - 4) % 3 == 0, "Depth must be 3 N + 4"

    # layers in each dense block
    nb_layers = int((depth - 4) / 3)

    # Initial convolution
    x = Convolution2D(nb_filter, (1, 11), kernel_initializer="he_uniform", padding="same", name="initial_conv2D", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(model_input)

    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay),
                            beta_regularizer=l2(weight_decay))(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate,
                                   weight_decay=weight_decay)
        # add transition_block
        x = transition_block(x, nb_filter, dropout_rate=dropout_rate, weight_decay=weight_decay)

    # The last dense_block does not have a transition_block
    x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate,
                               weight_decay=weight_decay)

    x = Activation('relu')(x)
    print x.shape
    x = GlobalAveragePooling2D()(x)
    x = keras.layers.core.Reshape((8,56))(x)
    x = Attention_layer()(x)
#     x = keras.layers.core.Reshape((448))(x)
    print x.shape
    x = Dense(nb_classes, activation='softmax', kernel_regularizer=l2(weight_decay), bias_regularizer=l2(weight_decay))(x)

    densenet = Model(inputs=model_input, outputs=x)

    if verbose: 
        print("DenseNet-%d-%d created." % (depth, growth_rate))

    return densenet

#define DenseNet parms
ROWS = 1
COLS = 18000
CHANNELS = 1
nb_classes = 9
batch_size = 10
nb_epoch = 40
img_dim = (ROWS,COLS,CHANNELS)
densenet_depth = 40
densenet_growth_rate = 12


model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim,depth=densenet_depth,
                  growth_rate = densenet_growth_rate)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

metric = AccF1MetricCallback()
earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')
checkpoint = keras.callbacks.ModelCheckpoint('DenseNet+A.h5', verbose=1, save_best_only=True)

xtrain = x_train.reshape(-1,1,18000,1)
xvalid = x_valid.reshape(-1,1,18000,1)
model.fit(xtrain, y_train, batch_size=32, epochs=500,
         validation_data=(xvalid,y_valid),
         callbacks = [ metric, checkpoint])

(?, 1, 4500, 448)
Tensor("attention_layer_4/div:0", shape=(?, 8, 56), dtype=float32)
Tensor("reshape_4/Reshape:0", shape=(?, 8, 56), dtype=float32)
Tensor("attention_layer_4/mul:0", shape=(?, 8, 56), dtype=float32)
(?, 56)
DenseNet-40-12 created.
Train on 5120 samples, validate on 1280 samples
Epoch 1/500
5120/5120 [==============================] - 155s 30ms/step - loss: 2.2930 - acc: 0.2244 - val_loss: 2.2799 - val_acc: 0.2523

Epoch 00001: val_loss improved from inf to 2.27990, saving model to DenseNet+A.h5
Epoch 2/500
5120/5120 [==============================] - 143s 28ms/step - loss: 2.2522 - acc: 0.2424 - val_loss: 2.2517 - val_acc: 0.2555

Epoch 00002: val_loss improved from 2.27990 to 2.25168, saving model to DenseNet+A.h5
Epoch 3/500
5120/5120 [==============================] - 144s 28ms/step - loss: 2.2409 - acc: 0.2451 - val_loss: 2.2409 - val_acc: 0.2547

Epoch 00003: val_loss improved from 2.25168 to 2.24087, saving model to DenseNet+A.h5
Epoch 4/500
5120/5120 [===========

5120/5120 [==============================] - 142s 28ms/step - loss: 2.0036 - acc: 0.3650 - val_loss: 2.1382 - val_acc: 0.3094

Epoch 00039: val_loss did not improve from 2.06241
Epoch 40/500
5120/5120 [==============================] - 144s 28ms/step - loss: 2.0000 - acc: 0.3703 - val_loss: 2.1484 - val_acc: 0.3406

Epoch 00040: val_loss did not improve from 2.06241
Epoch 41/500
5120/5120 [==============================] - 143s 28ms/step - loss: 1.9932 - acc: 0.3807 - val_loss: 2.4591 - val_acc: 0.2570

Epoch 00041: val_loss did not improve from 2.06241
Epoch 42/500
5120/5120 [==============================] - 144s 28ms/step - loss: 1.9923 - acc: 0.3844 - val_loss: 2.1738 - val_acc: 0.2984

Epoch 00042: val_loss did not improve from 2.06241
Epoch 43/500
5120/5120 [==============================] - 145s 28ms/step - loss: 1.9824 - acc: 0.3850 - val_loss: 2.0337 - val_acc: 0.3797

Epoch 00043: val_loss improved from 2.06241 to 2.03369, saving model to DenseNet+A.h5
Epoch 44/500
5120/5120 

5120/5120 [==============================] - 144s 28ms/step - loss: 1.8367 - acc: 0.4498 - val_loss: 1.9764 - val_acc: 0.4094

Epoch 00080: val_loss did not improve from 1.92978
Epoch 81/500
5120/5120 [==============================] - 144s 28ms/step - loss: 1.8292 - acc: 0.4500 - val_loss: 1.9355 - val_acc: 0.4289

Epoch 00081: val_loss did not improve from 1.92978
Epoch 82/500
5120/5120 [==============================] - 143s 28ms/step - loss: 1.8385 - acc: 0.4449 - val_loss: 1.9861 - val_acc: 0.4047

Epoch 00082: val_loss did not improve from 1.92978
Epoch 83/500
5120/5120 [==============================] - 143s 28ms/step - loss: 1.8284 - acc: 0.4498 - val_loss: 2.4565 - val_acc: 0.2250

Epoch 00083: val_loss did not improve from 1.92978
Epoch 84/500
5120/5120 [==============================] - 145s 28ms/step - loss: 1.8429 - acc: 0.4428 - val_loss: 1.9375 - val_acc: 0.4273

Epoch 00084: val_loss did not improve from 1.92978
Epoch 85/500
5120/5120 [==============================] - 


Epoch 00122: val_loss did not improve from 1.92284
Epoch 123/500
5120/5120 [==============================] - 145s 28ms/step - loss: 1.7355 - acc: 0.4766 - val_loss: 2.0768 - val_acc: 0.3320

Epoch 00123: val_loss did not improve from 1.92284
Epoch 124/500
5120/5120 [==============================] - 144s 28ms/step - loss: 1.7380 - acc: 0.4703 - val_loss: 1.9679 - val_acc: 0.4125

Epoch 00124: val_loss did not improve from 1.92284
Epoch 125/500
5120/5120 [==============================] - 144s 28ms/step - loss: 1.7374 - acc: 0.4740 - val_loss: 2.0506 - val_acc: 0.4070

Epoch 00125: val_loss did not improve from 1.92284
Epoch 126/500
5120/5120 [==============================] - 144s 28ms/step - loss: 1.7433 - acc: 0.4723 - val_loss: 1.9321 - val_acc: 0.4328

Epoch 00126: val_loss did not improve from 1.92284
Epoch 127/500
5120/5120 [==============================] - 145s 28ms/step - loss: 1.7316 - acc: 0.4729 - val_loss: 1.9771 - val_acc: 0.4125

Epoch 00127: val_loss did not improve f

5120/5120 [==============================] - 144s 28ms/step - loss: 1.7962 - acc: 0.4455 - val_loss: 2.0292 - val_acc: 0.3070

Epoch 00165: val_loss did not improve from 1.92269
Epoch 166/500
5120/5120 [==============================] - 146s 28ms/step - loss: 1.7449 - acc: 0.4613 - val_loss: 2.0588 - val_acc: 0.3289

Epoch 00166: val_loss did not improve from 1.92269
Epoch 167/500
5120/5120 [==============================] - 146s 28ms/step - loss: 1.7333 - acc: 0.4682 - val_loss: 1.9357 - val_acc: 0.4211

Epoch 00167: val_loss did not improve from 1.92269
Epoch 168/500
5120/5120 [==============================] - 146s 28ms/step - loss: 1.7078 - acc: 0.4734 - val_loss: 2.1525 - val_acc: 0.3211

Epoch 00168: val_loss did not improve from 1.92269
Epoch 169/500
5120/5120 [==============================] - 145s 28ms/step - loss: 1.7066 - acc: 0.4736 - val_loss: 1.9980 - val_acc: 0.4047

Epoch 00169: val_loss did not improve from 1.92269
Epoch 170/500
5120/5120 [=============================

...

5120/5120 [==============================] - 145s 28ms/step - loss: 1.6118 - acc: 0.5027 - val_loss: 2.0108 - val_acc: 0.4172

Epoch 00292: val_loss did not improve from 1.88082
Epoch 293/500
5120/5120 [==============================] - 144s 28ms/step - loss: 1.6214 - acc: 0.4982 - val_loss: 1.9794 - val_acc: 0.4266

Epoch 00293: val_loss did not improve from 1.88082
Epoch 294/500
5120/5120 [==============================] - 145s 28ms/step - loss: 1.6411 - acc: 0.4922 - val_loss: 1.9638 - val_acc: 0.4117

Epoch 00294: val_loss did not improve from 1.88082
Epoch 295/500
5120/5120 [==============================] - 144s 28ms/step - loss: 1.6117 - acc: 0.5055 - val_loss: 2.0027 - val_acc: 0.4133

Epoch 00295: val_loss did not improve from 1.88082
Epoch 296/500
5120/5120 [==============================] - 144s 28ms/step - loss: 1.6245 - acc: 0.4977 - val_loss: 1.9715 - val_acc: 0.4313

Epoch 00296: val_loss did not improve from 1.88082
Epoch 297/500
5120/5120 [=============================

5120/5120 [==============================] - 144s 28ms/step - loss: 1.5269 - acc: 0.5289 - val_loss: 2.2399 - val_acc: 0.3133

Epoch 00335: val_loss did not improve from 1.88082
Epoch 336/500
5120/5120 [==============================] - 143s 28ms/step - loss: 1.5184 - acc: 0.5279 - val_loss: 2.1194 - val_acc: 0.3953

Epoch 00336: val_loss did not improve from 1.88082
Epoch 337/500
5120/5120 [==============================] - 144s 28ms/step - loss: 1.5124 - acc: 0.5363 - val_loss: 2.0710 - val_acc: 0.3914

Epoch 00337: val_loss did not improve from 1.88082
Epoch 338/500
5120/5120 [==============================] - 145s 28ms/step - loss: 1.5087 - acc: 0.5328 - val_loss: 2.0359 - val_acc: 0.4242

Epoch 00338: val_loss did not improve from 1.88082
Epoch 339/500
5120/5120 [==============================] - 144s 28ms/step - loss: 1.5308 - acc: 0.5225 - val_loss: 3.3869 - val_acc: 0.1633

Epoch 00339: val_loss did not improve from 1.88082
Epoch 340/500
5120/5120 [=============================

5120/5120 [==============================] - 144s 28ms/step - loss: 1.4060 - acc: 0.5695 - val_loss: 3.8350 - val_acc: 0.1781

Epoch 00378: val_loss did not improve from 1.88082
Epoch 379/500
5120/5120 [==============================] - 143s 28ms/step - loss: 1.4055 - acc: 0.5639 - val_loss: 2.2217 - val_acc: 0.3906

Epoch 00379: val_loss did not improve from 1.88082
Epoch 380/500
5120/5120 [==============================] - 142s 28ms/step - loss: 1.4350 - acc: 0.5588 - val_loss: 2.3678 - val_acc: 0.3445

Epoch 00380: val_loss did not improve from 1.88082
Epoch 381/500
5120/5120 [==============================] - 144s 28ms/step - loss: 1.4084 - acc: 0.5668 - val_loss: 2.2584 - val_acc: 0.4031

Epoch 00381: val_loss did not improve from 1.88082
Epoch 382/500
5120/5120 [==============================] - 143s 28ms/step - loss: 1.4206 - acc: 0.5643 - val_loss: 2.4712 - val_acc: 0.3133

Epoch 00382: val_loss did not improve from 1.88082
Epoch 383/500
5120/5120 [=============================

5120/5120 [==============================] - 144s 28ms/step - loss: 1.3167 - acc: 0.6025 - val_loss: 5.2812 - val_acc: 0.1750

Epoch 00421: val_loss did not improve from 1.88082
Epoch 422/500
5120/5120 [==============================] - 144s 28ms/step - loss: 1.3175 - acc: 0.6059 - val_loss: 5.3244 - val_acc: 0.1570

Epoch 00422: val_loss did not improve from 1.88082
Epoch 423/500
5120/5120 [==============================] - 144s 28ms/step - loss: 1.5705 - acc: 0.5189 - val_loss: 3.1678 - val_acc: 0.2430

Epoch 00423: val_loss did not improve from 1.88082
Epoch 424/500
5120/5120 [==============================] - 144s 28ms/step - loss: 2.0835 - acc: 0.3289 - val_loss: 2.2963 - val_acc: 0.2352

Epoch 00424: val_loss did not improve from 1.88082
Epoch 425/500
5120/5120 [==============================] - 145s 28ms/step - loss: 1.8873 - acc: 0.4262 - val_loss: 2.0281 - val_acc: 0.3766

Epoch 00425: val_loss did not improve from 1.88082
Epoch 426/500
5120/5120 [=============================

5120/5120 [==============================] - 144s 28ms/step - loss: 1.2357 - acc: 0.6361 - val_loss: 3.5595 - val_acc: 0.1664

Epoch 00464: val_loss did not improve from 1.88082
Epoch 465/500
5120/5120 [==============================] - 145s 28ms/step - loss: 1.2667 - acc: 0.6332 - val_loss: 3.3172 - val_acc: 0.2188

Epoch 00465: val_loss did not improve from 1.88082
Epoch 466/500
5120/5120 [==============================] - 144s 28ms/step - loss: 1.3386 - acc: 0.6109 - val_loss: 2.5295 - val_acc: 0.3719

Epoch 00466: val_loss did not improve from 1.88082
Epoch 467/500
5120/5120 [==============================] - 144s 28ms/step - loss: 1.2868 - acc: 0.6188 - val_loss: 2.6238 - val_acc: 0.3773

Epoch 00467: val_loss did not improve from 1.88082
Epoch 468/500
5120/5120 [==============================] - 143s 28ms/step - loss: 1.2862 - acc: 0.6324 - val_loss: 2.5263 - val_acc: 0.2945

Epoch 00468: val_loss did not improve from 1.88082
Epoch 469/500
5120/5120 [=============================

In [33]:
temp = pd.DataFrame(y_train)
temp.describe()

,0,1,2,3,4,5,6,7,8
count,5120.00000,5120.000000,5120.000000,5120.000000,5120.000000,5120.000000,5120.000000,5120.000000,5120.000000
mean,0.14375,0.153711,0.107227,0.026758,0.236328,0.082617,0.094531,0.126172,0.028906
std,0.35087,0.360707,0.309431,0.161391,0.424868,0.275330,0.292595,0.332075,0.167559
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [34]:
temp = pd.DataFrame(y_valid)
temp.describe()

,0,1,2,3,4,5,6,7,8
count,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000
mean,0.142187,0.147656,0.107031,0.032813,0.252344,0.085156,0.096094,0.107813,0.028906
std,0.349379,0.354898,0.309274,0.178215,0.434527,0.279223,0.294835,0.310265,0.167609
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [35]:
xtrain.shape

(5120, 12, 18000, 1)

In [19]:
def conv_block(input, nb_filter, dropout_rate=0.5, weight_decay=1E-4):
    x = Activation('relu')(input)
    x = Convolution1D(nb_filter, (9), kernel_initializer="he_uniform", padding="same", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(x)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)
    return x

def dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=0.5, weight_decay=1E-4):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    feature_list = [x]

    for i in range(nb_layers):
        x = conv_block(x, growth_rate, dropout_rate, weight_decay)
        feature_list.append(x)
        x = Concatenate(axis=concat_axis)(feature_list)
        nb_filter += growth_rate

    return x, nb_filter

def transition_block(input, nb_filter, dropout_rate=0.5, weight_decay=1E-4):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    x = Convolution1D(nb_filter, (1), kernel_initializer="he_uniform", padding="same", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(input)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)
    x = AveragePooling1D((2), strides=(1))(x)

    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay),
                           beta_regularizer=l2(weight_decay))(x)

    return x

def createDenseNet(nb_classes, img_dim, depth=40, nb_dense_block=3, growth_rate=12, nb_filter=16, dropout_rate=0.5,
                     weight_decay=1E-4, verbose=True):

    model_input = Input(shape=img_dim)

    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    assert (depth - 4) % 3 == 0, "Depth must be 3 N + 4"

    # layers in each dense block
    nb_layers = int((depth - 4) / 3)

    # Initial convolution
    x = Convolution1D(nb_filter, (9), kernel_initializer="he_uniform", padding="same", name="initial_conv2D", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(model_input)

    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay),
                            beta_regularizer=l2(weight_decay))(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate,
                                   weight_decay=weight_decay)
        # add transition_block
        x = transition_block(x, nb_filter, dropout_rate=dropout_rate, weight_decay=weight_decay)

    # The last dense_block does not have a transition_block
    x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate,
                               weight_decay=weight_decay)

    x = Activation('relu')(x)
    print x.shape
#     x = AveragePooling2D((1, 25), strides=(1, 25))(x)
#     x = GlobalAveragePooling1D()(x)
#     x = keras.layers.core.Reshape((8,56))(x)
#     print x.shape
    x = Attention_layer()(x)
    x = Dense(nb_classes, activation='softmax', kernel_regularizer=l2(weight_decay), bias_regularizer=l2(weight_decay))(x)

    densenet = Model(inputs=model_input, outputs=x)

    if verbose: 
        print("DenseNet-%d-%d created." % (depth, growth_rate))

    return densenet



#define DenseNet parms
ROWS = 3
COLS = 18000
CHANNELS = 1
nb_classes = 9
batch_size = 32
nb_epoch = 40
img_dim = (ROWS,COLS)
densenet_depth = 40
densenet_growth_rate = 12

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim,depth=densenet_depth,
                  growth_rate = densenet_growth_rate)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')
checkpoint = keras.callbacks.ModelCheckpoint('DenseNet+A.h5', monitor='val_acc', verbose=1, save_best_only=True)

xtrain = x_train.reshape(-1,3,18000)
xvalid = x_valid.reshape(-1,3,18000)
model.fit(xtrain, y_train, batch_size=32, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [checkpoint])

(?, 1, 448)
Tensor("attention_layer_2/div:0", shape=(?, 1, 448), dtype=float32)
Tensor("activation_110/Relu:0", shape=(?, 1, 448), dtype=float32)
Tensor("attention_layer_2/mul:0", shape=(?, 1, 448), dtype=float32)
DenseNet-40-12 created.
Train on 5120 samples, validate on 1280 samples
Epoch 1/50
5120/5120 [==============================] - 20s 4ms/step - loss: 2.4962 - acc: 0.1748 - val_loss: 2.3292 - val_acc: 0.2375

Epoch 00001: val_acc improved from -inf to 0.23750, saving model to DenseNet+A.h5
Epoch 2/50
5120/5120 [==============================] - 12s 2ms/step - loss: 2.4048 - acc: 0.1998 - val_loss: 2.3049 - val_acc: 0.2422

Epoch 00002: val_acc improved from 0.23750 to 0.24219, saving model to DenseNet+A.h5
Epoch 3/50
5120/5120 [==============================] - 13s 2ms/step - loss: 2.3413 - acc: 0.2242 - val_loss: 2.2935 - val_acc: 0.2414

Epoch 00003: val_acc did not improve from 0.24219
Epoch 4/50
5120/5120 [==============================] - 12s 2ms/step - loss: 2.3125 - acc

KeyboardInterrupt: 